# Lab 1 - Module 3: Parameter Space Optimization

**Learning Objectives:**
- Optimize using only global error feedback
- Explore parameter space systematically
- Compare to gradient-based optimization

**Time:** ~15-20 minutes

---

**IMPORTANT:** Enter the same group code!

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import FloatSlider, Button, Output, HBox, VBox
from IPython.display import display

group_code = int(input("Enter your group code: "))
np.random.seed(group_code)

# Generate data
true_m = np.random.uniform(-3, 3)
true_b = np.random.uniform(-5, 5)
x = np.linspace(-5, 5, 25)
noise = np.random.normal(0, 1.0, size=len(x))
y = true_m * x + true_b + noise

print("✓ Data generated - ready for parameter space game!")

✓ Data generated - ready for parameter space game!


## 3.2 Optimizing Only on Global Error (MSE) – Parameter Space Game

In the last section, you could see both:

- the **data points**,
- and the **line** you were fitting,

plus local residuals and the global error (SSE).

In this section, we will **hide everything except the global error**.

We will work in **parameter space**:

- The horizontal axis will be the slope **m**.
- The vertical axis will be the intercept **b**.
- Each guess (m, b) corresponds to a line `y = m x + b` that could be fit to the data.

For each guess, the computer will:

1. Compute the **Mean Squared Error (MSE)** between your line and the data.
2. Add your guess (m, b, MSE) to a **table**.
3. Plot your guesses as **points in the (m, b) plane**, with the **color** of the most recent point showing its MSE (lower is better).

You will **not** see:

- the underlying data points,
- the true line,
- or the full error surface,

until the very end.

**Your goal:**  
Use only the global error (MSE) and the history of your guesses to move toward a good fit (a small MSE). This is directly analogous to what optimization algorithms do in machine learning: they “see” only the loss and adjust parameters to reduce it.

When your group feels you have done enough exploration, click the **“Done”** button. Then the notebook will reveal:

- a **color map** of MSE over the (m, b) grid,
- your guess history overlaid,
- and the location of the actual minimum.




In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, Button, Output, HBox, VBox
from IPython.display import display

# Make sure we have x, y, and sse or compute_mse defined earlier.

# Generate random "true" line parameters for your group,
# kept away from the slider edges.
true_m = np.random.uniform(-3.0, 3.0)
true_b = np.random.uniform(-3.0, 3.0)

# Range of m and b to explore
M_MIN, M_MAX = -5.0, 5.0
B_MIN, B_MAX = -5.0, 5.0

# Widgets: sliders for m and b, buttons, and output areas
m_slider = FloatSlider(
    description="m (slope)",
    min=M_MIN,
    max=M_MAX,
    step=0.1,
    value=0.0,
    continuous_update=False
)

b_slider = FloatSlider(
    description="b (intercept)",
    min=B_MIN,
    max=B_MAX,
    step=0.1,
    value=0.0,
    continuous_update=False
)

submit_button = Button(
    description="Submit guess",
    button_style="info"
)

done_button = Button(
    description="Done",
    button_style="success"
)

out_game = Output()
out_reveal = Output()

# Store guesses as a list of dicts
mse_history = []

def compute_mse(m, b):
    """Mean squared error for line y = m x + b against the data."""
    y_pred = m * x + b
    return float(np.mean((y - y_pred) ** 2))

def on_submit_clicked(b_widget):
    """Record a guess and update the table + (m,b) plot with colors = MSE."""
    m_guess = m_slider.value
    b_guess = b_slider.value
    mse_val = compute_mse(m_guess, b_guess)

    mse_history.append({
        "attempt": len(mse_history) + 1,
        "m": m_guess,
        "b": b_guess,
        "MSE": mse_val
    })

    df = pd.DataFrame(mse_history)

    with out_game:
        out_game.clear_output(wait=True)
        print("Parameter-space optimization: your goal is to find (m, b) with a small MSE.")
        print("You will only see global error (MSE), not the underlying data or line.\n")

        # Show recent guesses in a table
        print("Recent guesses:")
        display(df.tail(10))

        # Plot guesses in (m, b) space with colors tied to each point's MSE
        plt.figure(figsize=(7, 5))

        ms = df["m"].values
        bs = df["b"].values
        mses = df["MSE"].values

        # Color all points by MSE so color always reflects quality
        sc = plt.scatter(ms, bs,
                         c=mses,
                         cmap="viridis",
                         s=80,
                         edgecolor="black",
                         label="Your guesses")

        # Highlight the current best guess with a special marker
        best_idx = df["MSE"].idxmin()
        best_row = df.loc[best_idx]
        plt.scatter([best_row["m"]], [best_row["b"]],
                    marker="*",
                    s=200,
                    color="red",
                    edgecolor="black",
                    label="Best guess so far")

        plt.xlabel("m (slope)")
        plt.ylabel("b (intercept)")
        plt.xlim(M_MIN, M_MAX)
        plt.ylim(B_MIN, B_MAX)
        plt.title("Your guesses in (m, b) space\nColor = MSE (lower is better)")
        cbar = plt.colorbar(sc)
        cbar.set_label("MSE")
        plt.grid(True)
        plt.legend()
        plt.show()

def on_done_clicked(b_widget):
    """Show the full MSE landscape, global minimum, and the group's path."""
    submit_button.disabled = True
    done_button.disabled = True
    m_slider.disabled = True
    b_slider.disabled = True

    if not mse_history:
        with out_reveal:
            out_reveal.clear_output()
            print("No guesses were made. Nothing to reveal.")
        return

    # Compute MSE over a grid in (m, b) space
    m_vals = np.linspace(M_MIN, M_MAX, 80)
    b_vals = np.linspace(B_MIN, B_MAX, 80)
    M, B = np.meshgrid(m_vals, b_vals)
    mse_grid = np.zeros_like(M)

    for i in range(M.shape[0]):
        for j in range(M.shape[1]):
            mse_grid[i, j] = compute_mse(M[i, j], B[i, j])

    # Find approximate global minimum on the grid
    flat_index = np.argmin(mse_grid)
    i_min, j_min = np.unravel_index(flat_index, mse_grid.shape)
    m_min = M[i_min, j_min]
    b_min = B[i_min, j_min]

    # Compute least-squares solution from the data
    A = np.vstack([x, np.ones_like(x)]).T
    m_ls, b_ls = np.linalg.lstsq(A, y, rcond=None)[0]

    df = pd.DataFrame(mse_history)
    best_idx = df["MSE"].idxmin()
    best_guess = df.loc[best_idx]

    with out_reveal:
        out_reveal.clear_output()
        print("MSE landscape in (m, b) space with your guesses and the minima:\n")

        plt.figure(figsize=(8, 6))
        cs = plt.contourf(M, B, mse_grid, levels=30, cmap="viridis")
        cbar = plt.colorbar(cs)
        cbar.set_label("MSE")

        # Overlay all guesses, colored by their MSE
        sc2 = plt.scatter(df["m"], df["b"],
                          c=df["MSE"],
                          cmap="viridis",
                          s=80,
                          edgecolor="black",
                          label="Your guesses")

        # Highlight your best guess
        plt.scatter([best_guess["m"]], [best_guess["b"]],
                    color="red", marker="*",
                    s=200, edgecolor="black",
                    label="Your best guess")

        # Overlay approximate global minimum on grid
        plt.scatter([m_min], [b_min], color="white", marker="o",
                    s=150, edgecolor="black",
                    label="Grid global minimum")

        # Overlay least-squares solution
        plt.scatter([m_ls], [b_ls], color="yellow", marker="X",
                    s=150, edgecolor="black",
                    label="Least-squares solution")

        plt.xlabel("m (slope)")
        plt.ylabel("b (intercept)")
        plt.xlim(M_MIN, M_MAX)
        plt.ylim(B_MIN, B_MAX)
        plt.title("MSE Landscape and Your Path in (m, b) Space")
        plt.legend()
        plt.grid(True)
        plt.show()

        print(f"Approximate global minimum on grid: m ≈ {m_min:.3f}, b ≈ {b_min:.3f}")
        print(f"Least-squares solution from data:    m ≈ {m_ls:.3f}, b ≈ {b_ls:.3f}")
        print(f"True parameters used for your data:  m_true = {true_m:.3f}, b_true = {true_b:.3f}")
        print("\nYour best guess based on MSE only:")
        display(best_guess)

submit_button.on_click(on_submit_clicked)
done_button.on_click(on_done_clicked)

print("Parameter-space MSE game:")
print(f"- Adjust m and b using the sliders in ranges [{M_MIN}, {M_MAX}] and [{B_MIN}, {B_MAX}].")
print("- Click 'Submit guess' to record a new (m, b) and see its MSE.")
print("- Colors always represent MSE, so darker/“cooler” colors are better (lower error).")
print("- When your group is satisfied, click 'Done' to reveal the full MSE landscape.\n")

display(VBox([
    HBox([m_slider, b_slider]),
    HBox([submit_button, done_button]),
    out_game,
    out_reveal
]))


Parameter-space MSE game:
- Adjust m and b using the sliders in ranges [-5.0, 5.0] and [-5.0, 5.0].
- Click 'Submit guess' to record a new (m, b) and see its MSE.
- Colors always represent MSE, so darker/“cooler” colors are better (lower error).
- When your group is satisfied, click 'Done' to reveal the full MSE landscape.



In [3]:
# Summary
if mse_history:
    print(f"Total guesses: {len(mse_history)}")
    best_idx = min(range(len(mse_history)), key=lambda i: mse_history[i]['MSE'])
    print(f"Best MSE: {mse_history[best_idx]['MSE']:.4f}")
    print(f"Best (m, b): ({mse_history[best_idx]['m']:.2f}, {mse_history[best_idx]['b']:.2f})")
else:
    print("No guesses made yet!")

Total guesses: 16
Best MSE: 1.3466
Best (m, b): (1.30, -2.00)


## Next Steps

1. **Return to the LMS**
2. **Answer Questions 6-8** about parameter space optimization
3. **Continue to Module 4** for hidden function optimization